<a href="https://colab.research.google.com/github/WorldModelG15/final-task/blob/develop/train.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

ドライブをマウントして、マウントした場所にディレクトリ移動してから実行してください。



In [ ]:
# !git clone --recursive -b takata https://github.com/WorldModelG15/final-task.git
!git clone --recursive -b develop https://github.com/WorldModelG15/final-task.git
%cd final-task
%cd gym-duckietown/
!pip install -e .
%cd ..

In [ ]:
!pip install pyvirtualdisplay
!apt-get install python-opengl -y
!apt install xvfb -y
!apt-get install x11-utils -y
from pyvirtualdisplay import Display
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
%load_ext tensorboard

# This code creates a virtual display to draw game images on. 
# If you are running locally, just ignore it
import os
def create_display():
    display = Display(visible=0, size=(1400, 900))
    display.start()
    if type(os.environ.get("DISPLAY")) is not str or len(os.environ.get("DISPLAY"))==0:
        !sh xvfb start
        %env DISPLAY=:1
create_display()

In [ ]:
import sys
sys.path.append('/content/final-task/gym-duckietown/src/')
from utils.env import launch_env
from utils.wrappers import (
    ImgWrapper,
    DtRewardWrapper,
    ActionWrapper,
    ResizeWrapper,
    OriginalWrapper,
)
from dreamer.config import DreamerConfig
from dreamer.trainer import Trainer
import torch

 ### デフォルトのマップで学習する場合
env = launch_env(map_name="loop_pedestrians") # randomize_maps_on_reset=True でマップランダム化, enable_newly_visited_tile_reward=True でタイル報酬の有効化です

### オリジナルのマップで学習する場合
# map_dir_abs_path = (
#     "/root/mnt/final-task/gym-duckietown/created_maps/"  # ここは環境によって変えます
# )
# map_name = "zigzag"  # 'zigzag','oneloop','three_statics','loop_pedestrian'から選択です
# env = launch_env(
#     is_original_map=True, map_abs_path=map_dir_abs_path + map_name + ".yaml"
# )

env = ResizeWrapper(env)
env = ImgWrapper(env)  # to make the images from 120x160x3 into 3x120x160
env = ActionWrapper(env)
env = DtRewardWrapper(env)
env = OriginalWrapper(env)

device = torch.device("cuda") if torch.cuda.is_available() else torch.device("cpu")
config = DreamerConfig()
trainer = Trainer(env, device, config)

trainer.train()

In [ ]:
%tensorboard --logdir='./runs'